In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

# قراءة البيانات
df = pd.read_csv(r"E:\projects of camp\Machine Failure Prediction\data\data.csv")
x = df.drop(columns=['fail'])  # استبدل 'target' باسم العمود الهدف عندك
y = df['fail']

# توحيد البيانات
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)
#حفظscaler
import joblib
joblib.dump(scaler, "scaler.pkl")


# تصفية القيم الأقل من 0.2
x_scaled_df = pd.DataFrame(x_scaled, columns=x.columns)
df_filtered = x_scaled_df[(x_scaled_df >= 0.2).all(axis=1)]
x_filtered = df_filtered.values
y_filtered = y[df_filtered.index].values



# تقسيم البيانات
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# تحويل إلى Tensors
x_train_tensor = torch.tensor(x_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1)
x_test_tensor = torch.tensor(x_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).unsqueeze(1)

train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=20)

# تعريف النموذج
class SimpleNN(nn.Module):
    def __init__(self, input_dim,hidden_dim,dropout_rate):
        
        super().__init__()
        self.net= nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()
        )
     
    def forward(self, x):
        x = self.net(x)
        return x

def train_model(params):
    
    model = SimpleNN(x_train.shape[1], params['hidden_dim'], params['dropout_rate'])
    loss_fn = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=params['learning_rate'])

# التدريب
    for epoch in range(20):
        model.train()
        running_loss = 0.0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_fn(outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}/20, Loss: {running_loss/len(train_loader):.4f}")
# التقييم
    model.eval()

    with torch.no_grad():
        test_pred = model(x_test_tensor)
        test_loss = loss_fn(test_pred, y_test_tensor)
        test_accuracy = ((test_pred > 0.5).float() == y_test_tensor).float().mean().item()
        print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")
    # حفظ النموذج
    torch.save(model.state_dict(), "model.pth")
    return model, test_accuracy, test_loss.item()

train_model({'learning_rate': 0.001, 'hidden_dim': 32, 'dropout_rate': 0.2})     
# حفظ النموذج
        
    


Epoch 1/20, Loss: 3.7503
Epoch 2/20, Loss: 3.4624
Epoch 3/20, Loss: 3.5237
Epoch 4/20, Loss: 3.3076
Epoch 5/20, Loss: 2.4157
Epoch 6/20, Loss: 2.4154
Epoch 7/20, Loss: 2.7072
Epoch 8/20, Loss: 2.3938
Epoch 9/20, Loss: 1.8912
Epoch 10/20, Loss: 1.9440
Epoch 11/20, Loss: 1.7846
Epoch 12/20, Loss: 1.5350
Epoch 13/20, Loss: 1.5265
Epoch 14/20, Loss: 1.6533
Epoch 15/20, Loss: 1.9826
Epoch 16/20, Loss: 1.9888
Epoch 17/20, Loss: 1.5107
Epoch 18/20, Loss: 1.8580
Epoch 19/20, Loss: 1.4819
Epoch 20/20, Loss: 1.1437
Test Loss: 1.7861, Test Accuracy: 0.8730


(SimpleNN(
   (net): Sequential(
     (0): Linear(in_features=9, out_features=32, bias=True)
     (1): ReLU()
     (2): Dropout(p=0.2, inplace=False)
     (3): Linear(in_features=32, out_features=1, bias=True)
     (4): Sigmoid()
   )
 ),
 0.8730158805847168,
 1.7860684394836426)

In [3]:
# train.py
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from model import SimpleNN

# قراءة البيانات
df = pd.read_csv("data/data.csv")
x = df.drop(columns=['fail'])
y = df['fail']

# توحيد البيانات
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)
#حفظscaler
import joblib
joblib.dump(scaler, "scaler.pkl")


# تصفية القيم الأقل من 0.2
x_scaled_df = pd.DataFrame(x_scaled, columns=x.columns)
df_filtered = x_scaled_df[(x_scaled_df >= 0.2).all(axis=1)]
x_filtered = df_filtered.values
y_filtered = y[df_filtered.index].values



# تقسيم البيانات
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# تحويل إلى Tensors
x_train_tensor = torch.tensor(x_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1)
x_test_tensor = torch.tensor(x_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).unsqueeze(1)

train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=20)

# تدريب الموديل
input_dim = x_train.shape[1]
hidden_dim = 32
dropout_rate = 0.2
model = SimpleNN(input_dim, hidden_dim, dropout_rate)

loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(20):
    model.train()
    total_loss = 0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}")

# حفظ النموذج مع المعلمات
torch.save({
    "model_state": model.state_dict(),
    "input_dim": input_dim,
    "hidden_dim": hidden_dim,
    "dropout_rate": dropout_rate
}, "model.pth")

import joblib
joblib.dump(scaler, "scaler.pkl")



Epoch 1, Loss: 4.5250
Epoch 2, Loss: 4.6829
Epoch 3, Loss: 4.2917
Epoch 4, Loss: 3.3844
Epoch 5, Loss: 3.4080
Epoch 6, Loss: 3.8799
Epoch 7, Loss: 3.3197
Epoch 8, Loss: 2.9960
Epoch 9, Loss: 3.3501
Epoch 10, Loss: 2.5274
Epoch 11, Loss: 2.7671
Epoch 12, Loss: 2.0924
Epoch 13, Loss: 3.0280
Epoch 14, Loss: 2.7977
Epoch 15, Loss: 2.1366
Epoch 16, Loss: 2.9659
Epoch 17, Loss: 2.1642
Epoch 18, Loss: 3.3132
Epoch 19, Loss: 3.2987
Epoch 20, Loss: 2.0859


['scaler.pkl']

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [8]:
 import mlflow
 import mlflow.pytorch
 def objective(trail):
    params = {
        'learning_rate': trail.suggest_float('learning_rate', 1e-5, 1e-2, log=True),
        'hidden_dim': trail.suggest_int('hidden_dim', 16, 128),
        'dropout_rate': trail.suggest_float('dropout_rate', 0.0, 0.5)
    }
    with mlflow.start_run():
        model, accuracy, loss = train_model(params)
        mlflow.log_params(params)
        mlflow.log_metric('accuracy', accuracy)
        mlflow.log_metric('loss', loss)
        mlflow.pytorch.log_model(model, "model")
    
    
    return {'loss': loss, 'accuracy': accuracy} 

In [9]:
import optuna
mlflow.set_experiment("Machine_Failure_Prediction_Experiment")
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

2025/05/25 20:09:58 INFO mlflow.tracking.fluent: Experiment with name 'Machine_Failure_Prediction_Experiment' does not exist. Creating a new experiment.
[I 2025-05-25 20:09:58,191] A new study created in memory with name: no-name-f2b3a99e-e84f-4d11-a720-5b12bb59c281


Epoch 1/20, Loss: 9.4632
Epoch 2/20, Loss: 9.2322
Epoch 3/20, Loss: 6.9854
Epoch 4/20, Loss: 8.2357
Epoch 5/20, Loss: 7.2062
Epoch 6/20, Loss: 7.4016
Epoch 7/20, Loss: 8.4265
Epoch 8/20, Loss: 6.6240
Epoch 9/20, Loss: 5.6556
Epoch 10/20, Loss: 7.6108
Epoch 11/20, Loss: 7.0809
Epoch 12/20, Loss: 7.3234
Epoch 13/20, Loss: 7.1578
Epoch 14/20, Loss: 7.3387
Epoch 15/20, Loss: 6.8678
Epoch 16/20, Loss: 7.7966
Epoch 17/20, Loss: 6.8346
Epoch 18/20, Loss: 6.4648
Epoch 19/20, Loss: 6.3336
Epoch 20/20, Loss: 6.3103
Test Loss: 3.0247, Test Accuracy: 0.6614


2025/05/25 20:11:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
[W 2025-05-25 20:11:02,008] Trial 0 failed with parameters: {'learning_rate': 1.4216225635382324e-05, 'hidden_dim': 67, 'dropout_rate': 0.4308145404676525} because of the following error: The value {'loss': 3.0246710777282715, 'accuracy': 0.6613756418228149} could not be cast to float.
[W 2025-05-25 20:11:02,009] Trial 0 failed with value {'loss': 3.0246710777282715, 'accuracy': 0.6613756418228149}.


Epoch 1/20, Loss: 3.9911
Epoch 2/20, Loss: 4.2513
Epoch 3/20, Loss: 3.8419
Epoch 4/20, Loss: 3.5618
Epoch 5/20, Loss: 3.7255
Epoch 6/20, Loss: 3.6920
Epoch 7/20, Loss: 3.8524
Epoch 8/20, Loss: 3.9475
Epoch 9/20, Loss: 3.6986
Epoch 10/20, Loss: 3.9384
Epoch 11/20, Loss: 3.7926
Epoch 12/20, Loss: 3.0586
Epoch 13/20, Loss: 3.2652
Epoch 14/20, Loss: 3.8375
Epoch 15/20, Loss: 3.1879
Epoch 16/20, Loss: 3.0917
Epoch 17/20, Loss: 3.2610
Epoch 18/20, Loss: 3.2070
Epoch 19/20, Loss: 3.5804
Epoch 20/20, Loss: 3.7657
Test Loss: 3.7551, Test Accuracy: 0.5926


2025/05/25 20:11:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
[W 2025-05-25 20:11:18,925] Trial 1 failed with parameters: {'learning_rate': 2.432622407860758e-05, 'hidden_dim': 16, 'dropout_rate': 0.166597823943608} because of the following error: The value {'loss': 3.755086898803711, 'accuracy': 0.5925925970077515} could not be cast to float.
[W 2025-05-25 20:11:18,926] Trial 1 failed with value {'loss': 3.755086898803711, 'accuracy': 0.5925925970077515}.


...

2025/05/25 20:14:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
[W 2025-05-25 20:14:58,086] Trial 17 failed with parameters: {'learning_rate': 0.0002521378039173443, 'hidden_dim': 74, 'dropout_rate': 0.14826512704875877} because of the following error: The value {'loss': 2.1432838439941406, 'accuracy': 0.8571428656578064} could not be cast to float.
[W 2025-05-25 20:14:58,086] Trial 17 failed with value {'loss': 2.1432838439941406, 'accuracy': 0.8571428656578064}.


Epoch 1/20, Loss: 9.2663
Epoch 2/20, Loss: 9.9780
Epoch 3/20, Loss: 8.1318
Epoch 4/20, Loss: 6.2704
Epoch 5/20, Loss: 4.9588
Epoch 6/20, Loss: 6.2298
Epoch 7/20, Loss: 5.5161
Epoch 8/20, Loss: 3.7690
Epoch 9/20, Loss: 5.3276
Epoch 10/20, Loss: 4.2179
Epoch 11/20, Loss: 3.4648
Epoch 12/20, Loss: 3.4000
Epoch 13/20, Loss: 4.1328
Epoch 14/20, Loss: 4.0959
Epoch 15/20, Loss: 3.2589
Epoch 16/20, Loss: 3.0871
Epoch 17/20, Loss: 2.8795
Epoch 18/20, Loss: 3.5294
Epoch 19/20, Loss: 4.3471
Epoch 20/20, Loss: 3.2706
Test Loss: 3.2848, Test Accuracy: 0.8307


2025/05/25 20:15:09 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
[W 2025-05-25 20:15:09,745] Trial 18 failed with parameters: {'learning_rate': 0.0006250076825899478, 'hidden_dim': 122, 'dropout_rate': 0.45942388910746573} because of the following error: The value {'loss': 3.2847952842712402, 'accuracy': 0.8306878209114075} could not be cast to float.
[W 2025-05-25 20:15:09,745] Trial 18 failed with value {'loss': 3.2847952842712402, 'accuracy': 0.8306878209114075}.


Epoch 1/20, Loss: 7.5853
Epoch 2/20, Loss: 5.4854
Epoch 3/20, Loss: 4.6514
Epoch 4/20, Loss: 4.8103
Epoch 5/20, Loss: 3.5128
Epoch 6/20, Loss: 4.5341
Epoch 7/20, Loss: 4.1921
Epoch 8/20, Loss: 4.5984
Epoch 9/20, Loss: 5.0723
Epoch 10/20, Loss: 3.2137
Epoch 11/20, Loss: 3.9365
Epoch 12/20, Loss: 3.8625
Epoch 13/20, Loss: 3.8962
Epoch 14/20, Loss: 3.9362
Epoch 15/20, Loss: 3.8116
Epoch 16/20, Loss: 3.4768
Epoch 17/20, Loss: 2.5601
Epoch 18/20, Loss: 2.5133
Epoch 19/20, Loss: 2.4715
Epoch 20/20, Loss: 2.4390
Test Loss: 2.1005, Test Accuracy: 0.8466


2025/05/25 20:15:21 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
[W 2025-05-25 20:15:21,623] Trial 19 failed with parameters: {'learning_rate': 0.0008360583758808415, 'hidden_dim': 18, 'dropout_rate': 0.23967278060008024} because of the following error: The value {'loss': 2.1005446910858154, 'accuracy': 0.8465608358383179} could not be cast to float.
[W 2025-05-25 20:15:21,623] Trial 19 failed with value {'loss': 2.1005446910858154, 'accuracy': 0.8465608358383179}.
